In [121]:
import pandas as pd
import datetime
import time
import math

In [122]:
pd.set_option('float_format', '{:f}'.format)
folder = 'WF/16_12_2018_19_01 f'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [123]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [124]:
res_corr = res_eval.merge(res_adapt, left_on='instance', right_on='instance', how='inner')

In [125]:
res_corr = res_corr.fillna(0)
res_corr['diff_p'] = (res_corr['pred_p'] - res_corr['real_p'])
res_corr['diff_p'] = res_corr['diff_p']*res_corr['diff_p']

In [126]:
res_corr

,Unnamed: 0_x,instance,type_x,k_x,real_p,pred_p,Unnamed: 0_y,type_y,k_y,old_p,old_r,old_f,new_p,new_r,new_f,diff_p
0,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,0,IR,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,0.100531
1,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,1,SVDpp,0,0.510861,0.756943,0.463253,0.663371,0.544792,0.569446,0.100531
2,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,2,BPR,0,0.510861,0.756943,0.463253,0.510861,0.756943,0.463253,0.100531
3,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,3,GRU,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.100531
4,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,4,CNN,0,0.510861,0.756943,0.463253,0.496379,0.499669,0.498018,0.100531
5,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,5,DNN,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,0.100531
6,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,6,CRNN,0,0.510861,0.756943,0.463253,0.496360,0.497018,0.496689,0.100531
7,1,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SGD Regression,-,0.463253,2.618865,0,IR,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,4.646663
8,1,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SGD Regression,-,0.463253,2.618865,1,SVDpp,0,0.510861,0.756943,0.463253,0.663371,0.544792,0.569446,4.646663
9,1,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SGD Regression,-,0.463253,2.618865,2,BPR,0,0.510861,0.756943,0.463253,0.510861,0.756943,0.463253,4.646663


In [127]:
res_corr.groupby('type_x')[['pred_p', 'old_p', 'old_r', 'old_f']].corr()

pred_p     old_p  \
type_x                                                                  
CNN                                        pred_p  1.000000  0.113131   
                                           old_p   0.113131  1.000000   
                                           old_r   0.185680  0.507397   
                                           old_f   0.123284  0.875315   
CRNN                                       pred_p  1.000000  0.661182   
                                           old_p   0.661182  1.000000   
                                           old_r   0.132798  0.507397   
                                           old_f   0.840241  0.875315   
DNN                                        pred_p  1.000000  0.397757   
                                           old_p   0.397757  1.000000   
                                           old_r   0.178295  0.507397   
                                           old_f   0.303823  0.875315   
FeatureBased_Bayesian Ridge Regression     pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Lasso Regression              pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Linear Regression             pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Passive Aggressive Regression pred_p  1.000000 -0.263854   
                                           old_p  -0.263854  1.000000   
                                           old_r   0.063832  0.507397   
                                           old_f  -0.256556  0.875315   
FeatureBased_SGD Regression                pred_p  1.000000 -0.288787   
                                           old_p  -0.288787  1.000000   
                                           old_r   0.025122  0.507397   
                                           old_f  -0.285681  0.875315   
FeatureBased_SVM                           pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Theil-Sen Regression          pred_p  1.000000 -0.266372   
                                           old_p  -0.266372  1.000000   
                                           old_r   0.060080  0.507397   
                                           old_f  -0.259479  0.875315   
GRU                                        pred_p  1.000000  0.391153   
                                           old_p   0.391153  1.000000   
                                           old_r   0.019782  0.507397   
                                           old_f   0.354106  0.875315   

                                                     old_r     old_f  
type_x                                                                
CNN                                        pred_p 0.185680  0.123284  
                                           old_p  0.507397  0.875315  
                                           old_r  1.000000  0.543978  
                                           old_f  0.543978  1.000000  
CRNN                                       pred_p 0.132798  0.840241  
                                           old_p  0.507397  0.875315  
                                           old_r  1.000000  0.543978  
                                           old_f  0.543

In [128]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [129]:
res_adapt.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,9196.500000,0.000000,0.510357,0.572868,0.489646,0.120920,0.106809,0.041354,-0.389438,-0.466059,-0.448291
CNN,9198.500000,0.000000,0.510357,0.572868,0.489646,0.544594,0.554390,0.535658,0.034237,-0.018477,0.046012
CRNN,9200.500000,0.000000,0.510357,0.572868,0.489646,0.541660,0.546750,0.521264,0.031303,-0.026118,0.031618
DNN,9199.500000,0.000000,0.510357,0.572868,0.489646,0.544347,0.542798,0.535966,0.033989,-0.030070,0.046320
GRU,9197.500000,0.000000,0.510357,0.572868,0.489646,0.546142,0.572123,0.539621,0.035784,-0.000745,0.049975
IR,9194.500000,0.000000,0.510357,0.572868,0.489646,0.494517,0.500000,0.497238,-0.015840,-0.072868,0.007592
SVDpp,9195.500000,0.000000,0.510357,0.572868,0.489646,0.497057,0.502053,0.491530,-0.013300,-0.070815,0.001884


In [130]:
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str

In [131]:
res_adapt['new_f_max'] = res_adapt.groupby(['pair', 'type'])['new_f'].transform(max)
clean_a = res_adapt[res_adapt['new_f_max'] == res_adapt['new_f']]
clean_a.groupby('type').mean()
# res_adapt['diff_f_max'] = res_adapt.groupby(['pair'])['diff_f'].transform(max)
# clean_a = res_adapt[res_adapt['diff_f_max'] == res_adapt['diff_f']]
# clean_a.groupby('type').mean()
# clean_a.groupby('type').mean()
# grouped = clean_a[clean_a['type'] == 'CNN'].groupby(['type', 'pair']).mean().reset_index()
# grouped.sort_values('diff_f', ascending=False)

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f,new_f_max
type,,,,,,,,,,,,
BPR,8306.876712,0.000000,0.504827,0.594869,0.373800,0.499445,0.588302,0.367083,-0.005382,-0.006568,-0.006717,0.367083
CNN,3810.990050,0.000000,0.646211,0.740354,0.644477,0.729729,0.715264,0.706723,0.083518,-0.025090,0.062246,0.706723
CRNN,3964.323232,0.000000,0.651197,0.750034,0.646826,0.733820,0.713100,0.705204,0.082623,-0.036934,0.058378,0.705204
DNN,5368.113074,0.000000,0.593532,0.688327,0.579414,0.675956,0.641768,0.643702,0.082423,-0.046559,0.064288,0.643702
GRU,2773.130435,0.000000,0.700343,0.806495,0.714390,0.763683,0.799419,0.763293,0.063340,-0.007076,0.048903,0.763293
IR,9194.500000,0.000000,0.510357,0.572868,0.489646,0.494517,0.500000,0.497238,-0.015840,-0.072868,0.007592,0.497238
SVDpp,10312.840841,0.000000,0.495115,0.546978,0.480709,0.517126,0.518546,0.514840,0.022012,-0.028431,0.034131,0.514840


In [11]:
CRNN_adapt = res_adapt[res_adapt['type'] =='CRNN']

In [12]:
CRNN_adapt['new_f_max'] = CRNN_adapt.groupby(['pair'])['new_f'].transform(max)

C:\Users\shrag\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [14]:
CRNN_adapt_max.mean()

Unnamed: 0     1.040091e+03
k              0.000000e+00
old_p          5.242970e-03
old_r          5.000000e-01
old_f          1.034138e-02
new_p          9.988480e-01
new_r          9.213008e-01
new_f          9.552238e-01
diff_p         9.936051e-01
diff_r         4.213008e-01
diff_f         9.448824e-01
pair          1.000910e+278
new_f_max      9.552238e-01
dtype: float64

In [15]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['real_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['real_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [16]:
res_adapt_eval

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,real_e,pred_e,pred_e_max,k_max
0,0,1 AMC Token Path Ontobuilder MWBG,GRU_GRU,0,0.003618,0.5,0.007185,0.496382,0.500000,0.498184,0.5,4.999670e-01,0.5,0
1,1,1 AMC Token Path Ontobuilder MWBG,CNN_CNN,0,0.003618,0.5,0.007185,0.998678,0.818182,0.888227,0.5,7.877598e-01,0.5,0
2,2,1 AMC Token Path Ontobuilder MWBG,DNN_DNN,0,0.003618,0.5,0.007185,0.998678,0.818182,0.888227,0.5,5.000000e-01,0.5,0
3,3,1 AMC Token Path Ontobuilder MWBG,CRNN_CRNN,0,0.003618,0.5,0.007185,0.998678,0.818182,0.888227,0.5,4.999579e-01,0.5,0
4,4,1 AMC Token Path Ontobuilder MWBG,GRU_CNN,0,0.003618,0.5,0.007185,0.496382,0.500000,0.498184,0.5,7.877598e-01,0.5,1
5,5,1 AMC Token Path Ontobuilder MWBG,GRU_CNN,1,0.496382,0.5,0.498184,0.496382,0.500000,0.498184,0.5,9.178303e-01,0.5,1
6,6,1 AMC Token Path Ontobuilder MWBG,GRU_DNN,0,0.003618,0.5,0.007185,0.496382,0.500000,0.498184,0.5,5.000000e-01,0.5,0
7,7,1 AMC Token Path Ontobuilder MWBG,GRU_CRNN,0,0.003618,0.5,0.007185,0.496382,0.500000,0.498184,0.5,4.999579e-01,0.5,1
8,8,1 AMC Token Path Ontobuilder MWBG,GRU_CRNN,1,0.496382,0.5,0.498184,0.496382,0.500000,0.498184,0.5,4.999770e-01,0.5,1
9,9,1 AMC Token Path Ontobuilder MWBG,CNN_GRU,0,0.003618,0.5,0.007185,0.998678,0.818182,0.888227,0.5,4.999670e-01,0.5,0


In [17]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
BPR_FEATBayesian Ridge Regression,0.005235,0.500000,0.010323
BPR_FEATLasso Regression,0.005235,0.500000,0.010323
BPR_FEATLinear Regression,0.005235,0.500000,0.010323
BPR_FEATPassive Aggressive Regression,0.005235,0.500000,0.010323
BPR_FEATSGD Regression,0.005235,0.500000,0.010323
BPR_FEATSVM,0.005235,0.500000,0.010323
BPR_FEATTheil-Sen Regression,0.005235,0.500000,0.010323
BPR_GRU,0.005235,0.500000,0.010323
CNN_CNN,0.657181,0.610333,0.622833


In [18]:
res_adapt_eval['pair'], res_adapt_eval['alg'] = res_adapt_eval['instance'].str.split(' ', 1).str
CRNN_adapt_eval = res_adapt_eval[res_adapt_eval['type'] =='CRNN']

In [19]:
CRNN_adapt_eval['new_f_max'] = CRNN_adapt_eval.groupby(['pair'])['new_f'].transform(max)

In [20]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [21]:
CRNN_adapt_max

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f,pair,alg,new_f_max
13,13,1 Ontobuilder Term Match Ontobuilder Threshold,CRNN,0,0.003618,0.5,0.007185,0.999008,0.863636,0.920556,0.995390,0.363636,0.913371,1,Ontobuilder Term Match Ontobuilder Threshold,0.920556
27,27,10 WordNet Jiang Conrath Ontobuilder Threshold,CRNN,0,0.007143,0.5,0.014085,0.998967,0.928571,0.961021,0.991824,0.428571,0.946937,10,WordNet Jiang Conrath Ontobuilder Threshold,0.961021
41,41,100 Ontobuilder Term Match Ontobuilder Stable ...,CRNN,0,0.010766,0.5,0.021077,1.000000,1.000000,1.000000,0.989234,0.500000,0.978923,100,Ontobuilder Term Match Ontobuilder Stable Marr...,1.000000
83,83,103 WordNet Jiang Conrath Ontobuilder Threshold,CRNN,0,0.012856,0.5,0.025067,0.997188,0.892857,0.938590,0.984333,0.392857,0.913523,103,WordNet Jiang Conrath Ontobuilder Threshold,0.938590
118,118,104 WordNet Jiang Conrath Ontobuilder MWBG,CRNN,0,0.003106,0.5,0.006173,0.999376,0.900000,0.944132,0.996270,0.400000,0.937959,104,WordNet Jiang Conrath Ontobuilder MWBG,0.944132
125,125,104 WordNet Jiang Conrath Ontobuilder Threshold,CRNN,0,0.003106,0.5,0.006173,0.999376,0.900000,0.944132,0.996270,0.400000,0.937959,104,WordNet Jiang Conrath Ontobuilder Threshold,0.944132
153,153,105 Ontobuilder Term Match Ontobuilder Threshold,CRNN,0,0.006423,0.5,0.012683,0.999500,0.961538,0.979750,0.993077,0.461538,0.967067,105,Ontobuilder Term Match Ontobuilder Threshold,0.979750
181,181,106 AMC Token Path Ontobuilder Stable Marriage,CRNN,0,0.008333,0.5,0.016393,1.000000,1.000000,1.000000,0.991667,0.500000,0.983607,106,AMC Token Path Ontobuilder Stable Marriage,1.000000
195,195,107 Ontobuilder Term Match,CRNN,0,0.009649,0.5,0.018933,0.997331,0.863636,0.919715,0.987682,0.363636,0.900782,107,Ontobuilder Term Match,0.919715
223,223,108 WordNet Jiang Conrath Ontobuilder MWBG,CRNN,0,0.000757,0.5,0.001511,0.999697,0.800000,0.874848,0.998940,0.300000,0.873337,108,WordNet Jiang Conrath Ontobuilder MWBG,0.874848


In [22]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [23]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

FileNotFoundError: File b'adapt.csv' does not exist